In [1]:
import requests

import pandas as pd
import datetime

In [2]:
# Get symbols that user is tracking
names=["AAVE"]

In [3]:
def lookup(name):
    """Look up TVL for name"""

    # Contact API
    try:
        url = f"https://api.llama.fi/protocol/{name}"
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException:
        return None

    # Parse response
    try:
        quote = response.json()
        
        # Delete uneeded data
        chainTvls=quote["chainTvls"]
        for chain in chainTvls:
            chainTvls[chain]=chainTvls[chain]["tvl"]
                
        return {
            "name": quote["name"],
            "chainTvls":chainTvls,
            "tvl":quote["tvl"]
        }
    except (KeyError, TypeError, ValueError):
        return None

In [4]:
# Put all data into single dataframe
df_detailed = pd.DataFrame()

for name in names:
    data=lookup(name)

    for chain,dateTvl_dict in data["chainTvls"].items():
        current_df=pd.DataFrame.from_records(dateTvl_dict)
        current_df["protocol"]=data["name"]
        current_df["chain"]=chain
        df_detailed=pd.concat([df_detailed,current_df])
    
    current_df=pd.DataFrame.from_records(data["tvl"])
    current_df["protocol"]=data["name"]
    current_df["chain"]="actual tvl"
    df_detailed=pd.concat([df_detailed,current_df])

In [5]:
# Change column from int64 to datetime
df_detailed['date'] = pd.to_datetime(df_detailed['date'], unit='s')

# Sorting the index according to the protocol and chain
df_detailed=df_detailed.set_index(["protocol","chain","date"]).sort_index()

In [6]:
# Create a df with just the overview of the protocol
df_summary=df_detailed.loc[(slice(None),["pool2","staking","actual tvl","borrowed"]),:]

df_summary=df_summary.sort_index()

## Analysis of data
The code below will be used to convey the *weirdness* that I found out when working with the data. Mainly that the TVL of staking, borrowed and pool2 (liquidity pool) not adding up to the actual TVL.

In [7]:
# Cross-reference of data for summary
def cross_ref(df,dates):
    df=df.loc[(slice(None),slice(None),dates)]
    df=df.reset_index().set_index(["protocol","date","chain"])
    return df


# Returns value of borrowed + pool2 + staking and actual tvl
def compare(df):
    actualTVL=df.query("chain in ['actual tvl']").totalLiquidityUSD.sum()
    pool2BorrowStaking=df.query("chain not in ['actual tvl']").totalLiquidityUSD.sum()
    return actualTVL,pool2BorrowStaking
    

dates=["2021-10-03 00:00:00","2022-10-03 00:00:00","2022-10-15 00:00:00","2022-10-25 00:00:00"]

df=cross_ref(df_summary,dates)
for protocol, temp_df in df.groupby(["protocol","date"]):
    actualTVL,pool2BorrowStaking=compare(temp_df.loc[protocol])
    print(actualTVL-pool2BorrowStaking)

3099270658.166128
2054880915.9362092
1870765499.41544
1785904943.9741602


## Reason behind numbers not adding up
Looking into the data, we can see that for most chains (in the real sense of the word) like Polygon, there is only one component -- borrowed -- under it. However, the totalLiquidityUSD for Polygon vs Polygon-borrowed doesn't add up.

$ difference = actualtvl - staking - borrowed - pool2 $

This *difference* is also equivalent to the disparity between the chains and their borrowed component.

In [8]:
# At a glance we can see that besides the Ethereum chain with pool2 and staking, all other chains only
# contains a borrowed component
df_detailed.query("date in ['2022-10-03 00:00:00']")

totalLiquidityUSD
protocol chain              date                         
AAVE     Arbitrum           2022-10-03       2.951714e+07
         Arbitrum-borrowed  2022-10-03       1.078672e+07
         Avalanche          2022-10-03       7.339179e+08
         Avalanche-borrowed 2022-10-03       6.355419e+08
         Ethereum           2022-10-03       3.934549e+09
         Ethereum-borrowed  2022-10-03       1.548549e+09
         Ethereum-pool2     2022-10-03       8.687932e+07
         Ethereum-staking   2022-10-03       2.270213e+08
         Fantom             2022-10-03       2.113637e+06
         Fantom-borrowed    2022-10-03       6.876265e+05
         Harmony            2022-10-03       1.695390e+06
         Harmony-borrowed   2022-10-03       1.366466e+06
         Optimism           2022-10-03       4.467037e+08
         Optimism-borrowed  2022-10-03       7.824807e+08
         Polygon            2022-10-03       2.992404e+08
         Polygon-borrowed   2022-10-03       9.954322e+07
         actual tvl         2022-10-03       5.447737e+09
         borrowed           2022-10-03       3.078955e+09
         pool2              2022-10-03       8.687932e+07
         staking            2022-10-03       2.270213e+08

In [9]:
for today in dates:
    # Sum of all main chains
    all_chains=["Avalanche","Polygon","Optimism","Harmony","Arbitrum","Fantom","Ethereum"]
    total=df_detailed.query("date in @today and chain in @all_chains").totalLiquidityUSD.sum()

    # Sum of all sub chains
    suffix="-"
    
    if "chain" in df_detailed.index.names:
        df_detailed=df_detailed.reset_index("chain")
        
    df=df_detailed.query("date in @today")
    df=df.loc[df.chain.str.contains(suffix)]
    subchain_total=df.totalLiquidityUSD.sum()


    print(today)
    print("Difference:",(total-subchain_total))
    print("Significance:",((total-subchain_total)/total),"\n")



2021-10-03 00:00:00
Difference: 3099270658.16613
Significance: 0.21879003870969263 

2022-10-03 00:00:00
Difference: 2054880915.9362302
Significance: 0.3771990063541991 

2022-10-15 00:00:00
Difference: 1870765499.41541
Significance: 0.36725975313429693 

2022-10-25 00:00:00
Difference: 1785904943.9741402
Significance: 0.3587763293366404 



## Conclusion
For the dates tested, the significance of the untracked TVL ranges from around 21% to 37%. This is a signficant amount of TVL that is untracked and it would be important to understand how did this difference come about.